In [1]:
import plotly.express as px
import pandas as pd
import math

In [2]:
m_b = 2316272921462568965070799430859309430721132506041569198646776578261795862456408100512422166088475450870430901
m_s = 1502189842914195255446475418459104338923153704972451840000000000000000000000000000000000000000000000000000000
min_delta_b = 3111
min_delta_s = 1
max_delta_b = 8087
max_delta_s = 1000000000000000000000

In [3]:
perfect = []
best_imperfect = []
other_imperfect = []
delta_b_too_low = []
suboptimal_delta_b = []
prev_delta_b = None
best_imperfect_price = None
best_imperfect_interval = []
for delta_s in range(min_delta_s, max_delta_s + 1):
    delta_b = math.floor(delta_s * m_s / m_b)
    if (delta_b == 0):
        continue
    price = delta_s / delta_b
    if (delta_b > max_delta_b):
        delta_b = max_delta_b
    if (delta_b < min_delta_b):
        delta_b_too_low.append((delta_b, delta_s, price))
        continue
    if (delta_b == prev_delta_b):
        suboptimal_delta_b.append((delta_b, delta_s, price))
        continue
    prev_delta_b = delta_b
    if delta_b * m_b == delta_s * m_s:
      perfect.append((delta_b, delta_s, price))
      best_imperfect += best_imperfect_interval
      best_imperfect_price = None
      best_imperfect_interval = []
    elif (best_imperfect_price is None):
      best_imperfect_price = price
      best_imperfect_interval.append((delta_b, delta_s, price))
    elif (price == best_imperfect_price):
      best_imperfect_interval.append((delta_b, delta_s, price))
    elif (price < best_imperfect_price):
      best_imperfect_price = price
      other_imperfect += best_imperfect_interval
      best_imperfect_interval = [(delta_b, delta_s, price)]
    else:
      other_imperfect.append((delta_b, delta_s, price))
    if (delta_b == max_delta_b):
        break
    
best_imperfect += best_imperfect_interval

In [4]:
# Convert your data lists to Pandas DataFrames for easier handling
df1 = pd.DataFrame(perfect, columns=['delta_b', 'delta_s', 'price'])
df2 = pd.DataFrame(best_imperfect, columns=['delta_b', 'delta_s', 'price'])
df3 = pd.DataFrame(other_imperfect, columns=['delta_b', 'delta_s', 'price'])
df4 = pd.DataFrame(delta_b_too_low, columns=['delta_b', 'delta_s', 'price'])
df5 = pd.DataFrame(suboptimal_delta_b, columns=['delta_b', 'delta_s', 'price'])

# Combine the dataframes with an identifier column
df1['Series'] = 'Without rounding errors'
df2['Series'] = 'With rounding errors (best)'
df3['Series'] = 'With rounding errors (other)'
df4['Series'] = 'Delta_b too low'
df5['Series'] = 'Delta_b suboptimal'
combined_df = pd.concat([df1, df3, df2])

# Create an interactive scatter plot
fig = px.scatter(combined_df, x='delta_s', y='price', color='Series',
                 hover_data=['delta_b', 'delta_s', 'price'])

# Add plot titles and labels
fig.update_layout(title='effective price by delta selling',
                  xaxis_title='Delta_s',
                  yaxis_title='Price',
                  height=1600)

# fig.update_yaxes(range=[combined_df['price'].min(), combined_df['price'].max()])  # Adjust the range as needed

# # Sort the DataFrame by 'ratio' in descending order
# sorted_df = combined_df.sort_values(by='price', ascending=False)

# # Adding a line that connects the dots in decreasing order of their y-value
# fig.add_trace(px.line(sorted_df, x='delta_b', y='price').data[0])

# # Find the five lowest unique y-values (ratios)
# lowest_ratios = combined_df['price'].unique()
# lowest_ratios.sort()

# # Group by 'ratio' and draw lines for each of the five lowest ratios
# for price in lowest_ratios[:20]:
#     group = combined_df[combined_df['price'] == price]
#     # Sort each group by 'delta_b' to make line connections logical
#     sorted_group = group.sort_values(by='delta_b')
#     fig.add_trace(px.line(sorted_group, x='delta_b', y='price').data[0])

# # Adjusting the plot size and layout
# fig.update_layout(title='Scatter Plot with Lines for Lowest Y-values',
#                   xaxis_title='Delta_b',
#                   yaxis_title='Price',
#                   height=800)

# Show the plot
fig.show()


### Patterns:

- the (imperfect) optimal values appear to have equal distances in both deltas to each other 
  - -> if we're seeing the same value again, we can quit
- similarly for the points within a "slope"
- we can't see any way to compute those distances (see below for our attempts)
  - we could maybe try to measure them somehow:
    - start on the rightmost side
    - record the best price seen
    - go leftwards
    - if the price improves a second time, measure that distance
    - continue going leftwards, using that distance
    - as soon as the price gets worse, we have found the minimum of a "slope" (this is a global optimum in this chart, but not the other one)
    - we record this, and do the same thing again, this time also computing the distance between "slopes"
    - we could use this to efficiently search over the slopes, or simply be content with repeating this until we found a global optimum

In [5]:
11179 - 10940

239

In [6]:
11418 - 11179

239

In [7]:
7250 - 7095

155

In [8]:
7405 - 7250

155

In [9]:
8752 - 6564

2188

In [10]:
10940 - 8752

2188

In [11]:
5676 - 4257

1419

In [12]:
7095 - 5676

1419

In [13]:
def prime_factors(n):
    i = 2
    factors = {}
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors[i] = factors.get(i, 0) + 1
    if n > 1:
        factors[n] = factors.get(n, 0) + 1
    return factors

In [14]:
prime_factors(239)

{239: 1}

In [15]:
prime_factors(239 + 1)

{2: 4, 3: 1, 5: 1}

In [16]:
prime_factors(155)

{5: 1, 31: 1}

In [17]:
prime_factors(155 + 1)

{2: 2, 3: 1, 13: 1}

In [18]:
prime_factors(2188)

{2: 2, 547: 1}

In [19]:
prime_factors(2188 + 1)

{11: 1, 199: 1}

In [20]:
prime_factors(1419)

{3: 1, 11: 1, 43: 1}

In [21]:
prime_factors(1419 + 1)

{2: 2, 5: 1, 71: 1}

In [22]:
prime_factors(m_b)

{3: 220, 41: 1, 61: 1}

In [23]:
prime_factors(m_s)

{2: 222, 5: 55, 11: 1, 73: 1}